In [95]:
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix


In [2]:
df = pd.read_csv('College.csv')

In [3]:
summary_stats = []
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        mean = df[col].mean()
        std = df[col].std()
        summary_stats.append((col, f"{mean:.2f} ({std:.2f})"))
    elif col != 'Unnamed: 0':
        value_counts = df[col].value_counts()
        for value, count in value_counts.items():
            proportion = count / len(df)
            summary_stats.append((col, f"{value}  {count} ({proportion:.2f})"))

summary_df = pd.DataFrame(summary_stats, columns=['Column Name', 'Descriptive Stats'])
summary_df.to_excel('summary_stats.xlsx', index = False)


In [4]:
df.rename(columns={'Unnamed: 0': 'College Name'}, inplace=True)
df.columns

Index(['College Name', 'Private', 'Apps', 'Accept', 'Enroll', 'Top10perc',
       'Top25perc', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board',
       'Books', 'Personal', 'PhD', 'Terminal', 'S.F.Ratio', 'perc.alumni',
       'Expend', 'Grad.Rate'],
      dtype='object')

In [23]:
df['Acceptance Rate'] = (df['Accept'] / df['Apps'] ) * 100
df['Enrollment Rate'] = (df['Enroll'] / df['Accept'] ) * 100

In [43]:
enrollment_rate_df = df[['College Name', 'Acceptance Rate', 'Enrollment Rate']]
enrollment_rate_df['Competitiveness'] = pd.cut(enrollment_rate_df['Acceptance Rate'], bins = 5, right = False, labels = ['Highly Competitive', 'Very Competitive', 'Moderately Competitive', 'Less Competitive', 'Non-Competitive'])
# top_enrollment_rate_df = enrollment_rate_df.sort_values(by='Enrollment Rate', ascending=False).head(10)
fig = px.scatter(
    enrollment_rate_df,
    x = 'Enrollment Rate',
    y = 'Acceptance Rate',
    color = 'Competitiveness'
)
fig.write_image('Enrollment Rate vs Competitiveness.png')
fig.show()

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/338589762.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
top_10_top_25_df = df[['College Name', 'Acceptance Rate', 'Top10perc', 'Top25perc', 'Enrollment Rate']]
top_10_top_25_df['Perc Top 10 of Top 25'] = top_10_top_25_df['Top10perc'] / top_10_top_25_df['Top25perc']
top_10_top_25_df['Competitiveness'] = pd.cut(top_10_top_25_df['Acceptance Rate'], bins = 5, right = False, labels = ['Highly Competitive', 'Very Competitive', 'Moderately Competitive', 'Less Competitive', 'Non-Competitive'])
top_top_10_top_25_df = top_10_top_25_df.sort_values(by='Perc Top 10 of Top 25', ascending=False).groupby(by='Competitiveness').head(5)
fig = px.scatter(
    top_top_10_top_25_df,
    x = 'Perc Top 10 of Top 25',
    y = 'Enrollment Rate',
    color = 'Competitiveness'
)
fig.write_image('Enrollment Rate vs Percentage of Top 10 of Percentage of Top 25.png')
fig.show()

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/1266112411.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/1266112411.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/1266112411.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obser

In [45]:
grad_rate_df = df[['College Name', 'Top10perc', 'Top25perc', 'Acceptance Rate', 'Grad.Rate']]
grad_rate_df['Top 10 of Top 25'] = grad_rate_df['Top10perc'] / grad_rate_df['Top25perc']
grad_rate_df['Competitiveness'] = pd.cut(grad_rate_df['Acceptance Rate'], bins = 5, right = False, labels = ['Highly Competitive', 'Very Competitive', 'Moderately Competitive', 'Less Competitive', 'Non-Competitive'])
top_grad_rate_df = grad_rate_df.sort_values(by='Grad.Rate', ascending=False).groupby(by='Competitiveness').head(5)
fig = px.scatter(
    top_grad_rate_df,
    x = 'Grad.Rate',
    y = 'Top 10 of Top 25',
    color = 'Competitiveness'
)
fig.write_image('Grad Rate vs Percentage of Top 10 of Percentage of Top 25.png')
fig.show()

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/1648914093.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/1648914093.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/1648914093.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obser

In [91]:
model_df = df[['College Name', 'Grad.Rate', 'Acceptance Rate', 'Enrollment Rate','Top10perc', 'Top25perc']]
model_df['Top10percOfTop25perc'] = model_df['Top10perc'] / model_df['Top25perc']
model_df.drop(columns=['Top10perc', 'Top25perc'], inplace=True)
model_df['Competitiveness'] = pd.cut(model_df['Acceptance Rate'], bins = 5, right = False, labels = [1, 2, 3, 4, 5])
X = np.asarray(model_df[['Top10percOfTop25perc', 'Grad.Rate', 'Enrollment Rate']])
y = np.asarray(model_df['Competitiveness'].apply(lambda x: 1 if x == 1 else 0))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = sm.add_constant(X_train)

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/2384114118.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/2384114118.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5y/pjrsw60n65124xb_fjnr_68m0000gn/T/ipykernel_69937/2384114118.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [93]:
model = sm.GLM(y_train, X_train)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  543
Model:                            GLM   Df Residuals:                      539
Model Family:                Gaussian   Df Model:                            3
Link Function:               Identity   Scale:                       0.0068686
Method:                          IRLS   Log-Likelihood:                 583.81
Date:                Sun, 17 Nov 2024   Deviance:                       3.7022
Time:                        22:59:43   Pearson chi2:                     3.70
No. Iterations:                     3   Pseudo R-squ. (CS):            0.06945
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1087      0.022     -4.995      0.000      -0.151      -0.066
x1             0.1099      0.026      4.195      0.000       0.059       0.161
x2             0.0006      0.000      2.537      0.011       0.000       0.001
x3             0.0007      0.000      2.417      0.016       0.000       0.001
==============================================================================
"""

In [100]:
X_test = sm.add_constant(X_test)
y_pred = results.predict(X_test)

In [103]:
# Convert probabilities to binary predictions
threshold = 0.5
y_pred_class = (y_pred >= threshold).astype(int)

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_class)
precision = precision_score(y_test, y_pred_class)
recall = recall_score(y_test, y_pred_class)
specificity = tn / (tn + fp)  # True Negative Rate

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")

Accuracy: 0.97
Precision: 0.00
Recall: 0.00
Specificity: 1.00


/opt/homebrew/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



In [104]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

In [106]:
# Create the ROC curve plot
fig = go.Figure()

# Add ROC curve
fig.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.2f})',
    line=dict(color='blue')
))

# Add diagonal line for random chance
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random Chance',
    line=dict(dash='dash', color='gray')
))

# Customize layout
fig.update_layout(
    title='Receiver Operating Characteristic (ROC) Curve',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    xaxis=dict(range=[0.0, 1.0]),
    yaxis=dict(range=[0.0, 1.05]),
    legend=dict(x=0.8, y=0.2),
    template="plotly_white"
)

fig.write_image('ROC Curve.png')
# Show plot
fig.show()